In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)
data.test.cls = np.argmax(data.test.labels, axis=1)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [4]:
# Data Dimensions
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)
num_channels = 1
num_classes = 10

In [5]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

In [6]:
def cnn_model_fn(x):
    """Model function for CNN."""
    x_image = tf.reshape(x, [-1, 28, 28, 1])

    conv1 = tf.layers.conv2d(
        inputs=x_image,
        filters=16,
        kernel_size=[2, 2],
        padding="same",
        activation=tf.nn.relu
    )

    pool1 = tf.layers.max_pooling2d(
        inputs=conv1, 
        pool_size=[2, 2], 
        strides=2
    )

    conv2 = tf.layers.conv2d(
          inputs=pool1,
          filters=32,
          kernel_size=[2, 2],
          padding="same",
          activation=tf.nn.relu
    )
    
    pool2 = tf.layers.max_pooling2d(
        inputs=conv2, 
        pool_size=[2, 2], 
        strides=2
    )
    
    # flatten the layer
    layer_shape = pool2.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(pool2, [-1, num_features])
    
    # fully connect layer
    fc1 = tf.layers.dense(
        inputs=layer_flat, 
        units=256, 
        activation=tf.nn.relu
    )

    logits = tf.layers.dense(
        inputs=fc1, 
        units=num_classes
    )

    return logits

In [7]:
# cost function
y_pred = cnn_model_fn(x)

cost = tf.losses.mean_squared_error(
    labels=y_true,
    predictions=y_pred
)

# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)

# Calculate accuracy
y_pred_cls = tf.argmax(y_pred, 1)
correct_prediction = tf.equal(y_pred_cls, tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
sess = tf.Session()

In [11]:
sess.run(tf.global_variables_initializer())

In [12]:
train_batch_size = 100

def optimize(num_epochs):
    num_batches = len(data.train.labels)//100
    start_time = time.time()
    
    for i in range(num_epochs):
        train_acc = []
        for _ in range(num_batches):
            x_batch, y_true_batch = data.train.next_batch(train_batch_size)

            feed_dict_train = {
                x: x_batch,
                y_true: y_true_batch
            }
            batch_acc, _ = sess.run([accuracy, optimizer], feed_dict=feed_dict_train)
            train_acc.append(batch_acc)
        print("Epoch {}, Train accuracy: {:>6.1%}".format(i, np.mean(train_acc)))
        if i%5 == 0:
            print_test_accuracy()

    end_time = time.time()
    time_dif = end_time - start_time
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))
    
# Split the test-set into smaller batches of this size.
test_batch_size = 100

def print_test_accuracy():
    num_batches = len(data.test.labels)//100
    test_acc = []
    for _ in range(num_batches):
        x_batch, y_true_batch = data.test.next_batch(train_batch_size)
        feed_dict_test = {
            x: x_batch,
            y_true: y_true_batch
        }
        
        batch_acc = sess.run(accuracy, feed_dict=feed_dict_test)
        test_acc.append(batch_acc)
    print("Test accuracy: {:>6.1%}".format(np.mean(test_acc)))

In [13]:
print_test_accuracy()

Test accuracy:  12.8%


In [14]:
optimize(num_epochs=10)
print('='*30)
print_test_accuracy()

Epoch 0, Train accuracy:  95.0%
Test accuracy:  98.4%
Epoch 1, Train accuracy:  98.6%
Epoch 2, Train accuracy:  99.1%
Epoch 3, Train accuracy:  99.2%
Epoch 4, Train accuracy:  99.4%
Epoch 5, Train accuracy:  99.5%
Test accuracy:  99.1%
Epoch 6, Train accuracy:  99.6%
Epoch 7, Train accuracy:  99.7%
Epoch 8, Train accuracy:  99.7%
Epoch 9, Train accuracy:  99.8%
Time usage: 0:02:54
Test accuracy:  99.1%


In [ ]:
data.train.size

In [ ]:
dir(data.test)